In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)

In [ ]:
img = mnist.train.images[2]
plt.imshow(img.reshape((28, 28)), cmap='Greys_r')
plt.show()

In [ ]:
# building model 
lr = 0.001

x = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
y_ = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)

## building encoder
# 1st conv layer
# 28 x 28 x16
W_conv1 = tf.Variable(dtype=tf.float32, 
                 initial_value=tf.truncated_normal(shape=[5, 5, 1, 16], stddev=0.1))
b_conv1 = tf.Variable(dtype=tf.float32, 
                     initial_value=tf.constant(shape=[16], value=0.1))
conv2d_1 = tf.nn.conv2d(input=x, filter=W_conv1, strides=[1, 1, 1, 1], padding="SAME")
h_conv1 = tf.nn.relu(conv2d_1 + b_conv1)
# 14 x 14 x 16
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# 2nd conv layer
# 14 x 14 x 8
W_conv2 = tf.Variable(dtype=tf.float32, 
                      initial_value=tf.truncated_normal(shape=[5, 5, 16, 8], stddev=0.1))
b_conv2 = tf.Variable(dtype=tf.float32, 
                     initial_value=tf.constant(shape=[8], value=0.1))
conv2d_2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding="SAME")
h_conv2 = tf.nn.relu(conv2d_2 + b_conv2)
# 7 x 7 x 8
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# 3rd conv layer 
# 7 x 7 x8
W_conv3 = tf.Variable(dtype=tf.float32, 
                      initial_value=tf.truncated_normal(shape=[5, 5, 8, 8], stddev=0.1))
b_conv3 = tf.Variable(dtype=tf.float32, 
                      initial_value=tf.constant(shape=[8], value=0.1))
conv2d_3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding="SAME")
h_conv3 = tf.nn.relu(conv2d_3 + b_conv3)
# 4 x 4 x 8
encoded = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")



In [ ]:
## buildig decoder 
# upsample 1
# 7 x 7 x 8
upsample1 = tf.image.resize_nearest_neighbor(encoded, (7, 7))

# 1st conv layer
# 7 x 7 x 8 
d_W_conv1 = tf.Variable(dtype=tf.float32, 
                        initial_value=tf.truncated_normal(shape=[5, 5, 8, 8], stddev=0.1))
d_b_conv1 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.constant(shape=[8], value=0.1))
d_conv1 = tf.nn.conv2d(input=upsample1, filter=d_W_conv1, strides=[1, 1, 1, 1], padding="SAME")
d_h_conv1 = tf.nn.relu(d_conv1 + d_b_conv1)

# upsample 2
# 14 x 14 x 8 
upsample2 = tf.image.resize_nearest_neighbor(d_h_conv1, (14, 14))

# 2nd conv layer 
# 14 x 14 x 8
d_W_conv2 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.truncated_normal(shape=[5, 5, 8, 8], stddev=0.1))
d_b_conv2 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.constant(shape=[8], value=0.1))
d_conv2 = tf.nn.conv2d(input=upsample2, filter=d_W_conv2, strides=[1, 1, 1, 1], padding="SAME")
d_h_conv2 = tf.nn.relu(d_conv2 + d_b_conv2)

# upsample 3
# 28 x 28 x 8
upsample3 = tf.image.resize_nearest_neighbor(d_h_conv2, (28, 28))

# 3rd conv layer
# 28 x 28 x 16
d_W_conv3 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.truncated_normal(shape=[5, 5, 8, 16], stddev=0.1))
d_b_conv3 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.constant(shape=[16], value=0.1))
d_conv3 = tf.nn.conv2d(input=upsample3, filter=d_W_conv3, strides=[1, 1, 1, 1], padding="SAME")
d_h_conv3 = tf.nn.relu(d_conv3 + d_b_conv3)

# 4th conv layer
# 28 x 28 x 1
d_W_conv4 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.truncated_normal(shape=[5, 5, 16, 1], stddev=0.1))
d_b_conv4 = tf.Variable(dtype=tf.float32, 
                       initial_value=tf.constant(shape=[1], value=0.1))
d_conv4 = tf.nn.conv2d(input=d_h_conv3, filter=d_W_conv4, strides=[1, 1, 1, 1], padding="SAME")
logits = tf.nn.relu(d_conv4 + d_b_conv4)

decoded = tf.nn.sigmoid(logits, name="decoded")


In [ ]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=logits)
cross_entropy = tf.reduce_mean(loss)
train_step = tf.train.AdadeltaOptimizer(0.001).minimize(cost)

In [ ]:

## Training 
sess = tf.Session()

sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(100)
    imgs = batch[0].reshape((-1, 28, 28, 1))
    batch_cost, _ = sess.run([cross_entropy, train_step], 
                             feed_dict={x: imgs, y_: imgs})

    if i%200 == 0:
        print(batch_cost)